In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
import csv
import time

In [2]:
def scrape_page(url, search_terms, retries=3, timeout=5):
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout, verify=False)
            response.raise_for_status()  # Check if the request was successful

            if 'text/html' not in response.headers.get('Content-Type', ''):
                print(f"Skipping non-HTML content at: {url}")
                return []
            
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract contents of the accordions with class 'usa-accordion__content'
            accordions = soup.find_all('div', class_='usa-accordion__content')
            accordion_text = ' '.join(accordion.get_text(separator=' ') for accordion in accordions)
            
            page_text = (soup.get_text(separator=' ') + ' ' + accordion_text).lower()
            
            found_texts = []
            for term in search_terms:
                pattern = re.escape(term.lower())
                matches = re.findall(pattern, page_text)
                found_texts.extend(matches)
            
            return found_texts
        
        except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
            print(f"Error fetching {url}: {e}, attempt {attempt + 1} of {retries}")
            time.sleep(2)  # wait for 2 seconds before retrying

    return []

In [3]:
def get_all_urls(url, domain):
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = set()

    exclude_extensions = ['.pdf', '.docx', '.xlsx', '.csv', '.jpg', '.jpeg', '.png', '.gif', '.zip', '.rar', '.PDF']
    
    for link in soup.find_all('a', href=True):
        full_url = urljoin(url, link['href'])
        if urlparse(full_url).netloc == domain and not '#' in link['href']:
            if not any(full_url.endswith(ext) for ext in exclude_extensions):
                urls.add(full_url)
    
    return urls

In [4]:
def scrape_website(domain, search_terms):
    visited_urls = set()
    urls_to_visit = set([domain])
    results = []

    while urls_to_visit:
        current_url = urls_to_visit.pop()
        print(f"Visiting: {current_url}")
        if current_url not in visited_urls:
            visited_urls.add(current_url)
            found_texts = scrape_page(current_url, search_terms)
            if found_texts:
                results.append((current_url, found_texts))
            urls_to_visit.update(get_all_urls(current_url, urlparse(domain).netloc))
    
    return results

In [5]:
def export_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['URL', 'Found Text'])
        for url, texts in data:
            for text in texts:
                writer.writerow([url, text])

In [6]:
# Usage
if __name__ == "__main__":
    domain = "https://ohss.dhs.gov/"
    search_terms = ['Climate', 'Climate Migration', 'Climate Mitigation', 'Climate Adaptation', 'Climate Justice', 'Climate Resilience', 'Carbon Reductive', 'Carbon Footprint', 'Decarbonization', 'Electric Vehicles', 'Environmental justice', 'Emissions Reduction', 'Energy Efficiency', 'Fossil fuels', 'Global Warming', 'Greenhouse Gas', 'Greenhouse gas emissions', 'High performance buildings', 'Renewable Energy', 'Renewable Technology', 'solar', 'photovoltaics', 'wind power', 'thermal', 'nuclear', 'Sustainability', 'Water conservation'
]
    results = scrape_website(domain, search_terms)
    
    
    # Print results for debugging purposes
    print(results)
    
    # Export results to CSV
    export_to_csv(results, 'found_texts.csv')
    
    print("Results exported to found_texts.csv")

Visiting: https://ohss.dhs.gov/


C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Visiting: https://ohss.dhs.gov/about-our-data/geographic-region-definitions


C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Visiting: https://ohss.dhs.gov/khsm/fema-deployments


...

Visiting: https://ohss.dhs.gov/topics/immigration/immigration-enforcement/family-unit-actions-reports
Visiting: https://ohss.dhs.gov/khsm/uscg-search-and-rescue-responses
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table32
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2019/table17
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table33
Visiting: https://ohss.dhs.gov/topics/immigration/naturalizations/profiles-naturalized-citizens
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2016/table12


C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Visiting: https://ohss.dhs.gov/topics/screening
Visiting: https://ohss.dhs.gov/topics/immigration/lawful-permanent-residents/adjustments-lawful-permanent-residence-year-entry
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2014/table15
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2019/table1
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2011


C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\BRENT.JOHNSON\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ohss.dhs.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C

Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2019/table37
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table18
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2014/table24
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2017/table28
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2017/table33
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table5
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2017/table22
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2019/table35
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2009
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table21
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/yearbook-2016
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table39
Visiting: https://ohss.dhs.gov/topics/immigration/yearbook/2018/table25
Visiting: htt